In [34]:
import strawberryfields as sf
from strawberryfields.ops import *
from strawberryfields.utils import scale
from numpy import pi, sqrt
import matplotlib.pyplot as plt

# Photon number preservation with BS

In [82]:
eng, q = sf.Engine(2)
with eng:
    Fock(2)  | q[0]
    Fock(3)  | q[1]
    
    BSgate() | (q[0], q[1])
    Measure  | q[0]
    
eng.run("fock", cutoff_dim=6, reset_backend=False)

eng.reset_queue()  # resets the command queue
with eng:
    Measure | q[1]

state = eng.run("fock", cutoff_dim=6, reset_backend=False)

n_ = q[0].val
m_ = q[1].val
print(n_, ' + ', m_, ' = ', n_ + m_)

3  +  2  =  5


# HOM Interferometer

In [77]:
case1 = 0
case2 = 0
for i in range(1000):
    eng, q = sf.Engine(2)
    with eng:
        Fock(1)  | q[0]
        Fock(1)  | q[1]

        BSgate() | (q[0], q[1])
        Measure  | q[0]
        Measure  | q[1]

    state = eng.run("fock", cutoff_dim=6)
    n_ = q[0].val
    m_ = q[1].val
    if n_ == 2:
        case1 += 1
    elif m_ == 2:
        case2 += 1
    #print(n_, ' + ', m_, ' = ', n_ + m_)
    
print('mode 1 has 2 photon : ', case1)
print('mode 2 has 2 photon : ', case2)
    
print('Sum : ', case1 + case2)

mode 1 has 2 photon :  494
mode 2 has 2 photon :  506
Sum :  1000


# Measure with post selection

In [89]:
eng, q = sf.Engine(2)
with eng:
    Fock(2) | q[0]
    Fock(3) | q[1]
    BSgate() | (q[0], q[1])
    MeasureFock(select=0) | q[0] # select the 0 photon state
    Measure  | q[1]

eng.run("fock", cutoff_dim=6)
state = eng.run("fock", cutoff_dim=6, reset_backend=False)

n_ = q[0].val
m_ = q[1].val
print(n_, ' + ', m_, ' = ', n_ + m_)

0  +  5  =  5
